# Proyecto Heart Disease Prediction

## Análisis Exploratorio de Datos (EDA) y Preprocesamiento

**Objetivo**
Realizar un análisis completo de los datos para entender los factores de riesgo, identificar errores de calidad ("Data Health Check") y generar un dataset limpio (`interim`) listo para la fase de modelado.

**Metodología:**
1.  **Configuración de Arquitectura:** Definición de rutas absolutas para garantizar la reproducibilidad.
2.  **Data Health Check:** Auditoría técnica de nulos, duplicados y errores lógicos (ej. Colesterol = 0).
3.  **Limpieza (Interim):** Corrección de datos y persistencia del dataset saneado.
4.  **EDA Visual:** Análisis univariado, bivariado y multivariado usando gráficos interactivos.

---
* **Autor:** [Feliz Florian Jose Luis]
* **Fecha:** [12/11/2025]
* **Dataset:** data/01_raw/heart_disease_prediction_raw.csv
* **Tipo de Problema:** Clasificación Binaria (0: Normal vs 1: Enfermedad Cardíaca)
---

## 1. Configuración del Entorno y Arquitectura

Establecemos las librerías necesarias para la manipulación de datos y visualización.
Definimos la **Arquitectura de Rutas** basada en *Cookiecutter Data Science* para garantizar que el código sea ejecutable en cualquier entorno (Colab/Local) sin romper los enlaces.

In [17]:
# --- Librerías estándar ---
import pandas as pd
import numpy as np
import os
from google.colab import drive

# --- Librerías de visualización ---
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.io as pio

# --- CONFIGURACIÓN GLOBAL ---

# Estilo de gráficos profesional (Fondo blanco limpio)
pio.templates.default = "plotly_white"

# Montaje de Unidad (Persistencia en Google Drive)
drive.mount('/content/drive')

# --- DEFINICIÓN DE RUTAS (CONSTANTES) ---

# Ruta base del directorio del proyecto
PROJECT_DIR = "/content/drive/MyDrive/Colab Notebooks/heart_disease_prediction_mlops"

# Rutas de Ingesta y Procesamiento
RAW_PATH = os.path.join(PROJECT_DIR, "data", "01_raw", "heart_disease_prediction_raw.csv")
INTERIM_DIR = os.path.join(PROJECT_DIR, "data", "02_interim")
INTERIM_PATH = os.path.join(INTERIM_DIR, "heart_disease_prediction_cleaned.csv")

# Ruta de Reportes (Visualizaciones)
FIGURES_DIR = os.path.join(PROJECT_DIR, "data", "06_reporting", "figures")

# --- VALIDACIÓN DE DIRECTORIOS ---

# Creamos directorios si no existen
os.makedirs(INTERIM_DIR, exist_ok=True)
os.makedirs(FIGURES_DIR, exist_ok=True)

print("\n")
print(f"Entorno Configurado Correctamente.\n")
print(f"Origen (Raw): {RAW_PATH}\n")
print(f"Destino (Interim): {INTERIM_PATH}\n")
print(f"Reportes: {FIGURES_DIR}\n")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Entorno Configurado Correctamente.

Origen (Raw): /content/drive/MyDrive/Colab Notebooks/heart_disease_prediction_mlops/data/01_raw/heart_disease_prediction_raw.csv

Destino (Interim): /content/drive/MyDrive/Colab Notebooks/heart_disease_prediction_mlops/data/02_interim/heart_disease_prediction_cleaned.csv

Reportes: /content/drive/MyDrive/Colab Notebooks/heart_disease_prediction_mlops/data/06_reporting/figures



## 2. Carga y Data Health Check (Auditoría)

Cargamos los datos crudos y realizamos una inspección para detectar:
* Valores Nulos (`NaN`).
* Duplicados.
* **Errores de Lógica de Negocio:** En medicina, variables como la Presión Arterial (`RestingBP`) o el Colesterol (`Cholesterol`) no pueden ser 0. Si aparecen ceros, son datos perdidos encubiertos.

In [18]:
# Carga de Datos
try:
    df = pd.read_csv(RAW_PATH)
    print(f"Dataset cargado: {df.shape[0]} filas, {df.shape[1]} columnas")
except FileNotFoundError:
    print("ERROR CRÍTICO: No se encuentra el archivo raw. Verifica la ruta.")

# Inspección General
print("\n--- Información de Tipos de Datos ---\n")
df.info()

print("\n--- Estadísticas Descriptivas (Numéricas) ---\n")
display(df.describe().T)

# Auditoría de Errores Médicos (Lógica de Negocio)
print("\n--- Auditoría de Calidad de Datos ---\n")
zeros_Cholesterol = (df['Cholesterol'] == 0).sum()
zeros_RestingBP = (df['RestingBP'] == 0).sum()

print(f"Pacientes con Colesterol = 0 (Error): {zeros_Cholesterol}")
print(f"Pacientes con Presión Arterial = 0 (Error): {zeros_RestingBP}")

Dataset cargado: 918 filas, 12 columnas

--- Información de Tipos de Datos ---

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB

--- Estadísticas Descriptivas (Numéricas) ---



,count,mean,std,min,25%,50%,75%,max
Age,918.0,53.510893,9.432617,28.0,47.00,54.0,60.0,77.0
RestingBP,918.0,132.396514,18.514154,0.0,120.00,130.0,140.0,200.0
Cholesterol,918.0,198.799564,109.384145,0.0,173.25,223.0,267.0,603.0
FastingBS,918.0,0.233115,0.423046,0.0,0.00,0.0,0.0,1.0
MaxHR,918.0,136.809368,25.460334,60.0,120.00,138.0,156.0,202.0
Oldpeak,918.0,0.887364,1.066570,-2.6,0.00,0.6,1.5,6.2
HeartDisease,918.0,0.553377,0.497414,0.0,0.00,1.0,1.0,1.0



--- Auditoría de Calidad de Datos ---

Pacientes con Colesterol = 0 (Error): 172
Pacientes con Presión Arterial = 0 (Error): 1


## 3. Limpieza y Persistencia (Capa Interim)

**Acciones de Corrección:**
1.  Transformamos los valores `0` en `Cholesterol` y `RestingBP` a `NaN`. Esto permite que en la fase de entrenamiento los algoritmos de imputación (rellenado) los traten correctamente en lugar de aprender un patrón falso.
2.  Guardamos el resultado en `data/02_interim`. Este archivo será la fuente de verdad para el entrenamiento del modelo.

In [19]:
# --- Corrección de Datos ---

# Reemplazamos ceros biológicamente imposibles por NaN
df['Cholesterol'] = df['Cholesterol'].replace(0, np.nan)
df['RestingBP'] = df['RestingBP'].replace(0, np.nan)

print(f"Corrección aplicada: Se han convertido los ceros a NaN.\n")

# Guardado en Interim (Punto de Control)
df.to_csv(INTERIM_PATH, index=False)
print(f"Dataset limpio guardado exitosamente en: {INTERIM_PATH}\n")

Corrección aplicada: Se han convertido los ceros a NaN.

Dataset limpio guardado exitosamente en: /content/drive/MyDrive/Colab Notebooks/heart_disease_prediction_mlops/data/02_interim/heart_disease_prediction_cleaned.csv



## 4. Análisis Exploratorio de Datos (EDA)

### 4.1 Distribución de la Variable Objetivo
Analizamos el balance de clases. Un desbalance severo requeriría técnicas como SMOTE, pero este dataset está relativamente equilibrado.

In [20]:
# Gráfico de Pastel (Donut Chart)
fig = px.pie(
    df,
    names='HeartDisease',
    title='<b>Distribución del Target (Enfermedad Cardíaca)</b>',
    color_discrete_map={0: '#2ecc71', 1: '#e74c3c'}, # Verde (Sano) / Rojo (Enfermo)
    hole=0.4
)

# Mostrar y Guardar
fig.show()
fig.write_html(os.path.join(FIGURES_DIR, "balance_clases.html"))

### 4.2 Factores de Riesgo Categóricos
Analizamos cómo influyen variables como el sexo, el tipo de dolor de pecho o la angina inducida por ejercicio.
* **Nota Clave:** Observar `ChestPainType = ASY` (Asintomático) y su relación con la enfermedad.

In [21]:
Category_columns = ['Sex', 'ChestPainType', 'ExerciseAngina', 'ST_Slope']

for col in Category_columns:
    fig = px.histogram(
        df,
        x=col,
        color="HeartDisease",
        barmode='group',
        title=f"<b>Impacto de {col} en la Enfermedad</b>",
        color_discrete_map={0: '#2ecc71', 1: '#e74c3c'},
        text_auto=True
    )
    fig.show()
    fig.write_html(os.path.join(FIGURES_DIR, f"distribucion_{col}.html"))

### 4.3 Distribución Numérica y Detección de Outliers
Utilizamos **Boxplots** para visualizar la mediana y dispersión de variables vitales.
* Buscamos diferencias claras en las medianas entre grupos sanos (0) y enfermos (1), especialmente en `Oldpeak` y `MaxHR`.

In [22]:
number_columns = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']

for col in number_columns:
    fig = px.box(
        df,
        x="HeartDisease",
        y=col,
        color="HeartDisease",
        title=f"<b>Distribución de {col} (Detectando Outliers)</b>",
        color_discrete_map={0: '#2ecc71', 1: '#e74c3c'},
        points="outliers"
    )
    fig.show()
    fig.write_html(os.path.join(FIGURES_DIR, f"boxplot_{col}.html"))

### 4.3 Análisis Detallado: Edad vs Enfermedad
Comenzamos el análisis numérico con la variable **Edad (`Age`)**, dado que es un factor de riesgo natural.
Utilizamos un gráfico de caja (Boxplot) mostrando **todos los puntos individuales** (`points="all"`). Esto nos permite ver no solo la mediana, sino la densidad de pacientes en cada rango de edad.

In [23]:
fig_age = px.box(
    df,
    x="HeartDisease",
    y="Age",
    color="HeartDisease",
    title="<b>Distribución de Edad según Diagnóstico (Detalle)</b>",
    color_discrete_map={0: '#2ecc71', 1: '#e74c3c'},
    points="all" # Muestra la dispersión real de los datos
)

fig_age.show()

# Guardar
fig_age.write_html(os.path.join(FIGURES_DIR, "boxplot_age_detailed.html"))

### 4.4 Distribución General y Detección de Outliers
Analizamos el resto de variables clínicas (`RestingBP`, `Cholesterol`, `MaxHR`, `Oldpeak`).
Utilizamos **Boxplots con Muescas (`notched=True`)**.
* **Interpretación:** Si las "muescas" (la cintura de la caja) de dos grupos no se solapan, hay una fuerte evidencia estadística de que sus medianas son diferentes.

In [24]:
# Lista de variables numéricas clínicas
number_columns = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']

print("--- Generando Boxplots Clínicos ---\n")

for col in number_columns:
    # Usamos Boxplot para ver medianas y outliers claramente
    fig = px.box(
        df,
        x="HeartDisease",
        y=col,
        color="HeartDisease",
        title=f"<b>Distribución de {col} según Diagnóstico</b>",
        color_discrete_map={0: '#2ecc71', 1: '#e74c3c'},
        points="outliers", # Solo mostramos los atípicos para limpiar la vista
        notched=True # Muestra visualmente la significancia estadística de la mediana
    )

    fig.show()

    # GUARDAR
    filename = f"boxplot_{col}.html"
    save_path = os.path.join(FIGURES_DIR, filename)
    fig.write_html(save_path)
    print(f"Guardado: {filename}\n")

--- Generando Boxplots Clínicos ---



Guardado: boxplot_Age.html



Guardado: boxplot_RestingBP.html



Guardado: boxplot_Cholesterol.html



Guardado: boxplot_MaxHR.html



Guardado: boxplot_Oldpeak.html



### 4.5 Análisis de Correlación (Heatmap)
Evaluamos la **Multicolinealidad** entre variables numéricas.
* **Objetivo:** Identificar si hay variables redundantes.
* **Escala:** Usamos la escala `RdBu` (Rojo-Azul). Rojo intenso indica correlación positiva fuerte; Azul intenso, negativa.

In [25]:
# Calcular matriz de correlación

# Convertimos el target a numérico (ya lo es) y calculamos correlación
corr_matrix = df.corr(numeric_only=True).round(2)

# Visualizar
fig_corr = px.imshow(
    corr_matrix,
    text_auto=True,
    aspect="auto",
    title="<b>Matriz de Correlación (Mapa de Calor)</b>",
    color_continuous_scale='RdBu_r', # Rojo es correlación positiva (peligro)
    origin='lower'
)

fig_corr.show()

# Guardar
save_path = os.path.join(FIGURES_DIR, "heatmap_correlacion.html")
fig_corr.write_html(save_path)

### 4.6 Análisis Multivariado
**Scatter 3D:** Para entender la interacción compleja entre Edad, Ritmo Cardíaco y Gravedad (`Oldpeak`).

In [26]:
# Usamos el valor absoluto (.abs()) porque para el tamaño nos importa la magnitud, no el signo.
# Sumamos +1 para que los valores de 0 no desaparezcan del mapa.
df['Oldpeak_Size'] = df['Oldpeak'].abs() + 1

fig_scatter = px.scatter(
    df,
    x="Age",
    y="MaxHR",
    color="HeartDisease",
    size="Oldpeak_Size", # Usamos la columna positiva para el tamaño
    hover_data=['Oldpeak'], # Pero mostramos el valor REAL al pasar el mouse
    title="<b>Análisis Multivariado: Edad vs MaxHR vs Oldpeak</b>",
    color_discrete_map={0: '#2ecc71', 1: '#e74c3c'},
    opacity=0.7
)

fig_scatter.show()
fig_scatter.write_html(os.path.join(FIGURES_DIR, "scatter_multivariado.html"))

## 5. Conclusiones del EDA

Tras el análisis visual y estadístico, destacamos los siguientes puntos clave para la fase de modelado:

1.  **Calidad de Datos:** Se identificaron y corrigieron valores de `0` en Colesterol y Presión Arterial. Estos deberán ser imputados (rellenados) en el pipeline de entrenamiento.
2.  **Factores Predictivos Fuertes:**
    * **Oldpeak (Depresión del ST):** Muestra una clara separación; valores altos correlacionan fuertemente con la enfermedad.
    * **MaxHR:** Pacientes enfermos tienden a no alcanzar frecuencias cardíacas altas.
    * **ExerciseAngina:** La presencia de angina al ejercitarse es un indicador casi determinante.
3.  **Balance:** El dataset está equilibrado, permitiendo el uso de métricas estándar como Accuracy y ROC-AUC.
4.  **Próximos Pasos:** Utilizar el archivo `heart_disease_prediction_cleaned.csv` generado para entrenar modelos de Clasificación (Logistic Regression, Support Vector Machine (SVM), Decision Tree, Random Forest).